In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.model_selection import KFold

### Подход 1: градиентный бустинг "в лоб"

`1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).`

In [2]:
df = pd.read_csv('./features.csv', index_col='match_id')
X = df.loc[:, 'start_time':'dire_first_ward_time']

In [3]:
X.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


`2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.`

In [4]:
empty_cols = pd.Series(X.count())
empty_cols[empty_cols != 97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

#### Обоснование, почему их значения могут быть пропущены:
    first_blood_time - событие first_blood не произошло в первые 5 минут игры
    radiant_bottle_time - bottle не был куплен командой radiant в первые 5 минут игры

`3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.`

In [5]:
X = X.fillna(0)
empty_cols_filled = pd.Series(X.count())
empty_cols_filled[empty_cols_filled != 97230]

Series([], dtype: int64)

`3. Какой столбец содержит целевую переменную? Запишите его название.`

In [6]:
y = df['radiant_win']

`4. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?`

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
kf = KFold(n_splits=5, shuffle=True)
n_trees = np.array([5, 10, 20, 30, 50, 100])  # количество деревьев n_estimators
GBS = pd.DataFrame(np.zeros([len(n_trees), 2]), columns=['roc', 'time'])  # таблица для хранения auc_roc и времени
roc = np.zeros(5)

In [8]:
for i in range(len(n_trees)):
    clf = GradientBoostingClassifier(n_estimators=n_trees[i])
    start_time = datetime.datetime.now()
    j = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_test)
        roc[j] = roc_auc_score(y_test, y_pred[:, 1])
        j += 1
    GBS['roc'][i] = roc.mean()
    GBS['time'][i] = datetime.datetime.now() - start_time

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [9]:
GBS.set_index(n_trees)

,roc,time
5,0.634527,0:00:14.805051
10,0.665760,0:00:24.010907
20,0.682089,0:00:37.687501
30,0.690278,0:00:54.179343
50,0.697730,0:01:30.584313
100,0.706762,0:03:17.109708


### Что указать в отчете
В отчете по данному этапу вы должны ответить на следующие вопросы:
1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
2. Как называется столбец, содержащий целевую переменную?
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

### Ответ
##### Признаки, имеющие пропуски среди своих значений:
first_blood_time               
first_blood_team               
first_blood_player1            
first_blood_player2            
radiant_bottle_time            
radiant_courier_time           
radiant_flying_courier_time    
radiant_first_ward_time        
dire_bottle_time               
dire_courier_time              
dire_flying_courier_time       
dire_first_ward_time       
##### Что могут означать пропуски в этих признаках:
first_blood_time - событие first_blood не произошло в первые 5 минут игры         
radiant_bottle_time - bottle не был куплен командой radiant в первые 5 минут игры

##### Как называется столбец, содержащий целевую переменную?
radiant_win

##### Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? 
0:00:54.179343
##### Какое качество при этом получилось?
0.690278

##### Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 
Да
##### Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
Использовать лишь часть объектов, убрать незначимые признаки, изменить параметры классификатора (learning_rate, max_depth и т.п.)

### Подход 2: логистическая регрессия

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

`1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?`

In [11]:
reg_param = np.array([0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])  # параметр регуляризации C
LR = pd.DataFrame(np.zeros([len(reg_param), 2]), columns=['roc', 'time'])  # таблица для хранения auc_roc и времени
scaler = StandardScaler()
scaler.fit(X)
X_normalized = scaler.transform(X)
X_normalized = pd.DataFrame(X_normalized, columns=X.columns, index = X.index)
X_normalized.head()
roc = np.zeros(5)

In [12]:
for i in range(len(reg_param)):
    clf = LogisticRegression(C=reg_param[i])
    start_time = datetime.datetime.now()
    j = 0
    for train_index, test_index in kf.split(X_normalized):
        X_train, X_test = X_normalized.iloc[train_index], X_normalized.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_test)
        roc[j] = roc_auc_score(y_test, y_pred[:, 1])
        j += 1
    LR['roc'][i] = roc.mean() 
    LR['time'][i] = datetime.datetime.now() - start_time

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [13]:
LR.set_index(reg_param)

,roc,time
0.0001,0.711221,0:00:05.989473
0.0010,0.716274,0:00:10.514719
0.0100,0.716402,0:00:14.027214
0.1000,0.716379,0:00:14.109184
1.0000,0.716284,0:00:13.825405
10.0000,0.716330,0:00:14.638598
100.0000,0.716519,0:00:15.335258
1000.0000,0.716042,0:00:14.670286
10000.0000,0.716401,0:00:13.696900


`2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?`

In [14]:
to_drop = ['lobby_type', 'r1_hero','r2_hero','r3_hero','r4_hero','r5_hero', 'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
X_norm_cropped = X_normalized.drop(to_drop, axis=1)

In [15]:
LR_cropped = pd.DataFrame(np.zeros([len(reg_param), 2]), columns=['roc', 'time'])  # таблица для хранения auc_roc и времени
for i in range(len(reg_param)):
    clf = LogisticRegression(C=reg_param[i])
    start_time = datetime.datetime.now()
    j = 0
    for train_index, test_index in kf.split(X_norm_cropped):
        X_train, X_test = X_norm_cropped.iloc[train_index], X_norm_cropped.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_test)
        roc[j] = roc_auc_score(y_test, y_pred[:, 1])
        j += 1
    LR_cropped['roc'][i] = roc.mean() 
    LR_cropped['time'][i] = datetime.datetime.now() - start_time

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
LR_cropped.set_index(reg_param)

,roc,time
0.0001,0.711332,0:00:04.443177
0.0010,0.716419,0:00:09.328287
0.0100,0.716463,0:00:12.180494
0.1000,0.716455,0:00:12.483075
1.0000,0.716530,0:00:12.575615
10.0000,0.716506,0:00:12.321550
100.0000,0.716674,0:00:12.502098
1000.0000,0.716362,0:00:12.569589
10000.0000,0.716269,0:00:12.125486


`3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).`

In [17]:
heroes = ['r1_hero', 'r2_hero','r3_hero','r4_hero','r5_hero', 'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
for i in heroes:
    print(X[i].nunique())

108
108
108
108
108
108
108
108
108
108


In [18]:
heroes_id = np.sort(X['r1_hero'].unique()) 
heroes_id == np.sort(X['r3_hero'].unique())

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

`4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.`

In [19]:
# N — количество различных героев в выборке
N = 112  # 108
X_pick = np.zeros((X.shape[0], N))
for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [20]:
X_heroes = pd.DataFrame(X_pick[:,heroes_id-1], columns=heroes_id, index = X.index)

In [21]:
X_with_heroes = pd.concat([X_norm_cropped, X_heroes], axis=1)

`5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?`

In [22]:
LR_heroes = pd.DataFrame(np.zeros([len(reg_param), 2]), columns=['roc', 'time'])  # таблица для хранения auc_roc и времени
for i in range(len(reg_param)):
    clf = LogisticRegression(C=reg_param[i])
    start_time = datetime.datetime.now()
    j = 0
    for train_index, test_index in kf.split(X_with_heroes):
        X_train, X_test = X_with_heroes.iloc[train_index], X_with_heroes.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_test)
        roc[j] = roc_auc_score(y_test, y_pred[:, 1])
        j += 1
    LR_heroes['roc'][i] = roc.mean() 
    LR_heroes['time'][i] = datetime.datetime.now() - start_time

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
LR_heroes.set_index(reg_param)

,roc,time
0.0001,0.724916,0:00:05.516355
0.0010,0.746189,0:00:11.200587
0.0100,0.751668,0:00:18.406057
0.1000,0.751771,0:00:24.791103
1.0000,0.751964,0:00:26.601545
10.0000,0.751739,0:00:26.451009
100.0000,0.751712,0:00:26.991636
1000.0000,0.751622,0:00:27.182207
10000.0000,0.751843,0:00:26.618543


`6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).`

In [32]:
df_test = pd.read_csv('./features_test.csv', index_col='match_id')
data_test = df_test.loc[:, 'start_time':'dire_first_ward_time']
data_test = data_test.fillna(0)
data_test_normalized = scaler.transform(data_test)
data_test_normalized = pd.DataFrame(data_test_normalized, columns=data_test.columns, index = data_test.index)
data_test_norm_cropped = data_test_normalized.drop(to_drop, axis=1)
N = 112  # 108
data_test_pick = np.zeros((data_test.shape[0], N))
for i, match_id in enumerate(data_test.index):
    for p in range(5):
        data_test_pick[i, data_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        data_test_pick[i, data_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
data_test_heroes = pd.DataFrame(data_test_pick[:,heroes_id-1], columns=heroes_id, index = data_test.index)
data_test_with_heroes = pd.concat([data_test_norm_cropped, data_test_heroes], axis=1)

C:\Users\Ruslan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [44]:
clf = LogisticRegression(C=1.0)
start_time = datetime.datetime.now()
clf.fit(X_with_heroes, y)
y_pred = clf.predict_proba(data_test_with_heroes)
T = datetime.datetime.now() - start_time

In [75]:
print(y_pred[:, 1].min(), y_pred[:, 1].max())

0.00845941516167 0.996430679815


In [65]:
answer = pd.concat([pd.Series(data_test_with_heroes.index), pd.Series(y_pred[:, 1])], axis=1)

In [70]:
answer.columns=['match_id', 'radiant_win']

In [74]:
answer.to_csv('kaggle_dota.csv', index=False)

##### Что указать в отчете
В отчете по данному этапу вы должны ответить на следующие вопросы:
1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
3. Сколько различных идентификаторов героев существует в данной игре?
4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

### Ответ
##### Какое качество получилось у логистической регрессии над всеми исходными признаками? 
0.7165
##### Как оно соотносится с качеством градиентного бустинга? 
Лучше
##### Чем вы можете объяснить эту разницу? 
Много признаков, разреженные данные
##### Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
Да
##### Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? 
0.7167 Практически не влияет
##### Чем вы можете объяснить это изменение?
Категориальный признак в своей исходной записи неинформативен
##### Сколько различных идентификаторов героев существует в данной игре?
108
##### Какое получилось качество при добавлении "мешка слов" по героям? 
0.752
##### Улучшилось ли оно по сравнению с предыдущим вариантом? 
Да
##### Чем вы можете это объяснить?
Признак выбора героев информативен в представлении "мешка слов"
##### Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
0.008 0.996